# Funny birds to Time Series

In [20]:
import os
import glob
import cv2
import json

from tqdm.auto import tqdm
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

In [21]:
TEST = True
CENTER = "CO" # I-> Img center, C->Contour, CO->Center from the original contour

## Cream màscares

In [22]:
def are_all_similar(a, b, c, tolerance=5):
    return (
        np.logical_and(abs(a - b) <= tolerance, np.logical_and(abs(a - c) <= tolerance, abs(b - c) <= tolerance))
    )

def get_mask(parts, tolerance=5):
    parts_np = np.array(parts)[:, :, :-1]
    parts_np_unic = parts_np.reshape(-1, parts_np.shape[2])
    parts_np_unic = np.unique(parts_np_unic, axis=0)
    parts_np_unic = parts_np_unic.astype(np.float64)
    diff = are_all_similar(parts_np_unic[:, 0], parts_np_unic[:, 1], parts_np_unic[:, 2], tolerance)

    values = parts_np_unic[diff]
    mask = np.zeros((256, 256), dtype=np.uint8)
    for val in values:
        if (val[0] == 170) or (val[0] == 255):
            continue
        mask = mask + np.logical_and.reduce((parts_np[:,:,0] ==  val[0], parts_np[:,:,1] == val[1], parts_np[:,:,2] == val[2]))
        
    return (1 - mask)

def get_time_series(mask, center=None):   
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnts = sorted(cnts, key=lambda x: x.size, reverse=True)

    multiple_cnts = (len(cnts) > 1) and (cnts[1].size > 500)
        
    cnt = cnts[0]

    if center is None:
        M = cv2.moments(cnt)
        center = int(M["m01"] / M["m00"]), int(M["m10"] / M["m00"])
    
    time_series = []

    for point in cnt:
        point = point[0]
        time_series.append(float(np.sqrt((point[0] - center[0])**2 + (point[1] - center[1])**2)))

    return time_series, center, multiple_cnts

In [23]:
if TEST:
    part_map_paths = "./data/funny_birds/v2/test_part_map"
    inter_map_paths = "./data/funny_birds/v2/test_interventions"
else:
    part_map_paths = "./data/funny_birds/v2/train_part_map"
    inter_map_paths = None

IMG_PATH = f"./{part_map_paths}/**/*.png"

In [24]:
images_paths = sorted(glob.glob(IMG_PATH))

classes = set()
for p in images_paths:
    classe = int(p.split(os.path.sep)[-2])

    if classe not in classes:
        classes.add(classe)

cls_dict = {}
for i, classe in enumerate(classes):
    cls_dict[classe] = i

In [25]:
if TEST:
    folder_name = "test"
else:
    folder_name = "train"

    
for i, img_path in enumerate(tqdm(images_paths)):
    path_parts = img_path.split(os.path.sep) 
    bird_class = path_parts[-2]
    bird_class_translated = cls_dict[int(bird_class)]
    
    file_img_name = path_parts[-1]
    img_name = file_img_name.split(".")[0]

    img_parts = Image.open(img_path)
    mask = get_mask(img_parts, 40)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3, 3),np.uint8))
    mask[mask != 0] = 1

    if mask.max() == 0:
        print(f"Image {img_path} empty")
        continue

    # Path of the mask
    out_path = os.path.join("output", f"{folder_name}_{CENTER}", str(bird_class_translated).zfill(3), img_name)
    os.makedirs(out_path, exist_ok=True)
    cv2.imwrite(f"./{out_path}/image.png", mask * 255)

    # To Time series
    center_img = mask.shape[0] // 2, mask.shape[1] // 2     

    center = None
    if CENTER == "I":
        center = center_img
    
    time_series, center_cnt, multiple_cnts = get_time_series(mask.astype(np.uint8))
    if multiple_cnts:
        print(f"Image {str(i).zfill(5)} multiple cnts")
        continue
        
    if CENTER == "CO": # Center of the first contour
        center = center_cnt
                                       
    with open(f"./{out_path}/time_series.json", "w") as final:
        	json.dump(time_series, final)
    
    # Saving interventions
    if inter_map_paths is not None:
        for intervetions_paths in tqdm(sorted(glob.glob(f"./{inter_map_paths}/{bird_class}/{img_name}/*.png")), leave=False):
            name_intervent = intervetions_paths.split(os.path.sep)[-1].split(".")[0]
            
            intervetion_img = Image.open(intervetions_paths)
            intervetion_img_np = np.array(intervetion_img)[:, :, :-1]
    
            logical_mask = 1 - np.logical_and.reduce((intervetion_img_np[:,:,0] == 85, intervetion_img_np[:,:,1] == 85, intervetion_img_np[:,:,2] == 153))
            logical_mask = mask * logical_mask
    
            time_series, _, _ = get_time_series(logical_mask.astype(np.uint8), center)
                    
            with open(f"./{out_path}/{name_intervent}.json", "w") as final:
            	json.dump(time_series, final)
    
            cv2.imwrite(f"./{out_path}/{name_intervent}.png", logical_mask * 255)

  0%|          | 0/470 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

...

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]